In [9]:
import yfinance as yf
import numpy as np
import pandas as pd
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
import statsmodels as sm 
from scipy.stats import gaussian_kde

class TimeSeriesModel:
    def __init__(self, time_series, significance_level=0.05):
        """
        Initialize the class with the time series data and significance level for model evaluation.
        
        :param time_series: pandas Series representing the time series.
        :param significance_level: The significance level for model evaluation.
        """
        self.time_series = time_series
        self.significance_level = significance_level
        self.model = None
        self.residuals = None
    
    def is_stationary(self):
        """
        Perform the Augmented Dickey-Fuller (ADF) test to check if the series is stationary.
        
        :return: True if the series is stationary, False otherwise.
        """
        adf_result = sm.tsa.adfuller(self.time_series)
        p_value = adf_result[1]
        return p_value < self.significance_level

    def difference(self, order=1):
        """
        Difference the series to make it stationary.
        
        :param order: Order of differencing (default is 1).
        :return: Differenced time series.
        """
        self.time_series = self.time_series.diff(order).dropna()
        return self.time_series
    
    def select_best_arima(self, max_p=5, max_d=2, max_q=5):
        """
        Automatically select the best ARIMA model using stepwise search via pmdarima's auto_arima.
        
        :param max_p: Maximum order of AR.
        :param max_d: Maximum differencing.
        :param max_q: Maximum order of MA.
        :return: Best ARIMA model based on AIC.
        """
        print("Performing stepwise search for best ARIMA model...")
        
        stepwise_model = pm.auto_arima(
              self.time_series,
            start_p=0, start_q=0,
            max_p=max_p, max_d=max_d, max_q=max_q,
            seasonal=False,
            trace=True,  # Print each iteration
            error_action='ignore',  # Ignore failed models
            suppress_warnings=True,
            stepwise=True
        )
        
        self.model = ARIMA(self.time_series, order=stepwise_model.order).fit()
        self.residuals = self.model.resid
        print(f"Selected ARIMA model: {stepwise_model.order} with AIC: {stepwise_model.aic()}")
        
        return self.model
    
    def simulate_forecast(self, num_simulations=10000, forecast_periods=10):
        """
        Simulate forecasts of the time series model.
        
        :param num_simulations: Number of forecast simulations.
        :param forecast_periods: Number of forecast periods ahead.
        :return: Simulated forecasts (2D array of size num_simulations x forecast_periods).
        """
        forecast_results = np.zeros((num_simulations, forecast_periods))
        
        # KDE for residuals to simulate noise matching the residual distribution
        kde = gaussian_kde(self.residuals)
        
        # Simulate forecasts
        for i in range(num_simulations):
            forecast_values = []
            noise_term = kde.resample(forecast_periods).flatten()  # Sample residuals as noise
            forecast = self.model.forecast(steps=forecast_periods)
            
            # Use .values to correctly access the forecast array
            forecast_values = forecast.values + noise_term  # Add sampled noise
            
            forecast_results[i, :] = forecast_values
        
        return forecast_results
    
    def percent_exceed(self, simulations, threshold):
        """
        Calculate the percentage of simulations that exceed a user-specified value in each forecast period.
        
        :param simulations: 2D array of forecast simulations (from simulate_forecast).
        :param threshold: Value that simulations are compared against.
        :return: Percent of simulations exceeding the threshold in each forecast period.
        """
        return np.mean(simulations > threshold, axis=0)


In [10]:

# Fetching Google stock data using yfinance (GOOGL is the ticker for Alphabet Inc)
google_stock = yf.download('GOOGL', start='2010-01-01', end='2020-12-31')

# Use the 'Close' price for time series modeling
time_series = google_stock['Close'].dropna()

# Initialize the time series model
ts_model = TimeSeriesModel(time_series)

# Check if the series is stationary
print("Is Stationary:", ts_model.is_stationary())

# If not stationary, difference the series
if not ts_model.is_stationary():
    ts_model.difference()

# Select and fit the best ARIMA model
best_model = ts_model.select_best_arima()
print("Best ARIMA Model Summary:")
print(best_model.summary())

# Simulate forecasts
simulations = ts_model.simulate_forecast(num_simulations=10000, forecast_periods=10)

# Calculate the percent of simulations exceeding a certain threshold
threshold_value = 1500  # Example threshold for Google stock price
percent_exceed = ts_model.percent_exceed(simulations, threshold=threshold_value)
print(f"Percent of simulations exceeding {threshold_value} in each period:")
print(percent_exceed)

[*********************100%***********************]  1 of 1 completed


Is Stationary: False
Performing stepwise search for best ARIMA model...
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=5958.076, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=5928.727, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=5930.009, Time=0.07 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=5930.124, Time=0.06 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=5930.274, Time=0.08 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=5931.518, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=5926.213, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=5956.445, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=5927.738, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=5927.845, Time=0.15 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=5927.433, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=5930.202, Time=0.41 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 1.184 seconds
Selected ARIMA m